In [ ]:
import torch
from torch.utils.data import Dataset
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim

class SinDataset(Dataset):
    def __init__(self, x, included_timesteps, shift):
        """
        Initialize the dataset with the data.

        Args:
            data (list or array-like): List of numbers to use as data.
        """
        self.data = x
        self.included_timesteps = included_timesteps
        self.shift = shift

    def __len__(self):
        """
        Return the number of samples in the dataset.
        """
        return len(self.data)-self.included_timesteps

    def __getitem__(self, idx):
        """
        Fetch the data and its label at index `idx`.

        Args:
            idx (int): Index of the sample to retrieve.
        
        Returns:
            tuple: (data, label) where label is the square of the data.
        """
        x = np.sin(self.data[idx:idx+self.included_timesteps] + self.shift)  # Get the input data
        y = np.sin(self.data[idx+self.included_timesteps] + self.shift)         # Label is the square of the input
        return x, y


## Loading data

In [126]:
timeline = np.linspace(0, 100, 10000)
x1 = SinDataset(timeline, 20, 0)
x2 = SinDataset(timeline, 20, 1)
dataloader1 = DataLoader(x1, batch_size=1)
dataloader2 = DataLoader(x2, batch_size=1)

In [127]:
for (x1, y1), (x2, y2) in zip(dataloader1, dataloader2):
    print(x1.shape, y1.shape, x1.shape, y1.shape)
    x = torch.stack((x1, x2), dim=2).float()
    y = torch.stack((y1, y2), dim=1).float()
    print(x.shape, y.shape)
    print(x.dtype, y)
    break

torch.Size([1, 20]) torch.Size([1]) torch.Size([1, 20]) torch.Size([1])
torch.Size([1, 20, 2]) torch.Size([1, 2])
torch.float32 tensor([[0.1987, 0.9320]])


## Setting up Network

In [128]:
class Net(nn.Module):
    def __init__(self, n_neurons, input_shape):
        super(Net, self).__init__()
        
        self.lstm1 = nn.LSTM(input_size=input_shape, hidden_size=n_neurons, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=input_shape, hidden_size=n_neurons, batch_first=True)

        self.fc1 = nn.Linear(n_neurons, n_neurons)
        self.fc2 = nn.Linear(n_neurons, n_neurons)
        self.fc = nn.Linear(n_neurons, 2)
    
    def forward(self, x):
        _, (h_n1, _) = self.lstm1(x[:, :, 0])
        _, (h_n2, _) = self.lstm2(x[:, :, 1])
        out = torch.cat((h_n1, h_n2))
        out = self.fc(out)
        return out

# Training loop

In [131]:
n_neurons = 12
included_timesteps = 20
epochs = 20

model = Net(n_neurons, included_timesteps)
loss_function = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_curve = []

for epoch in range(epochs):
    loss_total = 0
    model.train()

    for (x1, y1), (x2, y2) in zip(dataloader1, dataloader2):
        optimizer.zero_grad()
        x = torch.stack((x1, x2), dim=2).float()
        y = torch.stack((y1, y2), dim=1).float()

        predictions = model(x)
        loss = loss_function(predictions, y)
        loss_total += loss.item()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch}: {loss=}')


C:\Users\Max Tost\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([1, 2])) that is different to the input size (torch.Size([2, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0: loss=tensor(0.0024, grad_fn=<MseLossBackward0>)
Epoch 1: loss=tensor(0.0300, grad_fn=<MseLossBackward0>)


KeyboardInterrupt: 